In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 02: Cleaning and Chunking\n",
    "\n",
    "## Goal\n",
    "Clean text data and create semantic chunks for vector embedding."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 1: Imports\n",
    "import json\n",
    "import os\n",
    "import re\n",
    "import pandas as pd\n",
    "from pathlib import Path\n",
    "\n",
    "print(\"✅ Libraries imported\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 2: Load raw documents\n",
    "raw_docs = []\n",
    "for file in os.listdir(\"../data/raw\"):\n",
    "    if file.endswith(\".json\") and file != \"summary.json\":\n",
    "        with open(f\"../data/raw/{file}\", \"r\") as f:\n",
    "            raw_docs.append(json.load(f))\n",
    "\n",
    "print(f\"📚 Loaded {len(raw_docs)} documents\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 3: Text cleaning function\n",
    "def clean_text(text):\n",
    "    \"\"\"Clean and normalize text\"\"\"\n",
    "    # Lowercase\n",
    "    text = text.lower()\n",
    "    # Remove extra whitespace\n",
    "    text = re.sub(r'\\s+', ' ', text)\n",
    "    # Remove special characters but keep basic punctuation\n",
    "    text = re.sub(r'[^\\w\\s.,!?-]', '', text)\n",
    "    # Normalize units\n",
    "    text = re.sub(r'(\\d+)\\s*(mmhg|bpm|hr|ms)', r'\\1 \\2', text)\n",
    "    return text.strip()\n",
    "\n",
    "# Test cleaning\n",
    "test_text = \"Blood pressure   should be below 140/90 mmHg!!!\"\n",
    "print(f\"Test cleaning: {clean_text(test_text)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 4: Chunking function\n",
    "def chunk_text(text, chunk_size=300, overlap=50):\n",
    "    \"\"\"Split text into overlapping chunks\"\"\"\n",
    "    words = text.split()\n",
    "    chunks = []\n",
    "    \n",
    "    if len(words) <= chunk_size:\n",
    "        return [\" \".join(words)]\n",
    "    \n",
    "    for i in range(0, len(words), chunk_size - overlap):\n",
    "        chunk = \" \".join(words[i:i + chunk_size])\n",
    "        chunks.append(chunk)\n",
    "        \n",
    "        # Stop if we've reached the end\n",
    "        if i + chunk_size >= len(words):\n",
    "            break\n",
    "            \n",
    "    return chunks\n",
    "\n",
    "# Test chunking\n",
    "test_long_text = \" \".join([\"word\" + str(i) for i in range(500)])\n",
    "test_chunks = chunk_text(test_long_text, chunk_size=100, overlap=20)\n",
    "print(f\"Test chunking created {len(test_chunks)} chunks\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 5: Process all documents\n",
    "cleaned_chunks = []\n",
    "chunk_id = 0\n",
    "\n",
    "for doc in raw_docs:\n",
    "    cleaned_text = clean_text(doc[\"text\"])\n",
    "    chunks = chunk_text(cleaned_text, chunk_size=300, overlap=50)\n",
    "    \n",
    "    for chunk in chunks:\n",
    "        cleaned_chunks.append({\n",
    "            \"chunk_id\": f\"chunk_{chunk_id}\",\n",
    "            \"source\": doc[\"source\"],\n",
    "            \"title\": doc[\"title\"],\n",
    "            \"text\": chunk,\n",
    "            \"word_count\": len(chunk.split())\n",
    "        })\n",
    "        chunk_id += 1\n",
    "\n",
    "print(f\"✅ Created {len(cleaned_chunks)} chunks from {len(raw_docs)} documents\")\n",
    "print(f\"Average chunk length: {sum([c['word_count'] for c in cleaned_chunks])/len(cleaned_chunks):.0f} words\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 6: Save cleaned chunks\n",
    "with open(\"../data/cleaned/chunks.json\", \"w\") as f:\n",
    "    json.dump(cleaned_chunks, f, indent=2)\n",
    "\n",
    "# Also save as CSV for easy viewing\n",
    "df_chunks = pd.DataFrame(cleaned_chunks)\n",
    "df_chunks.to_csv(\"../data/cleaned/chunks.csv\", index=False)\n",
    "\n",
    "print(\"\\n📊 Chunk Statistics:\")\n",
    "print(df_chunks['source'].value_counts())\n",
    "print(f\"\\n✅ Saved {len(cleaned_chunks)} chunks to data/cleaned/chunks.json and chunks.csv\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cell 7: Preview chunks\n",
    "print(\"\\n🔍 Sample Chunks:\")\n",
    "for i, chunk in enumerate(cleaned_chunks[:3]):\n",
    "    print(f\"\\nChunk {i+1} ({chunk['source']} - {chunk['title']}):\")\n",
    "    print(f\"{chunk['text'][:150]}...\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.12.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined